In [28]:
%run monoalphabetic_substitution.ipynb

## Histogram下的频率分析

首先咱们尝试最简单的方案,即简单的比较密文(Cipher Text)当中字母的频率与大数据结果的差异,并通过类似于$\chi^2$test的方法来评判

通过简单的频率分析后的密钥解密出来的结果与正常英文材料之间的差异.并通过调整密钥中字母以及符号的排列顺序使得结果更好.

### 0:准备

**0.1** 这一步首先LETTERS里面确认key包含的符号,也就是哪些符号需要参与替换

In [44]:
LETTERS = 'abcdefghijklmnopqrstuvwxyz'

In [45]:
#因为后面很多函数都需要将文字标准化(统一大小写，去掉标点空格等非字母符号，去掉换行符号)
def StandardizeText(text):
    text = text.lower()
    text = text.replace('\n','')
    newText = []
    for letter in text:
        if letter.isalpha():
            newText.append(letter)
        
    return ''.join(newText)

**0.2** 这一步需要建立letterCount的值为0的字典用于之后计算频数,频率以及排序

### 1:建立计算频数并排序的函数HistogramFreqExtract

In [46]:
import numpy as np
from itertools import repeat

In [47]:
def HistogramFreqExtract(message):
# 0.每次提取都要让leterCount归零
    letterCount = dict(zip(list(LETTERS),list(repeat(0,26))))
# 1.算出文章中字母的个数
    message = StandardizeText(message) #先标准化
    AlphaNumber = 0
    for letter in message:
        if letter in LETTERS:
            AlphaNumber += 1
# 2.算出字母的频数
    for letter in message:   
        if letter in LETTERS:
            letterCount[letter] += 1 
# 3.算出字母的频率

    MessageCount = list(letterCount.values())
    MessageFreqArray = (np.asarray(MessageCount)/AlphaNumber)*100
    MessageFreqList = MessageFreqArray.tolist()# numpy中的函数,能把array变成list
    DictLetterFreq = dict(zip(list(letterCount.keys()),MessageFreqList)) #两个list通过zip()变成有序对,然后dict()变成字典 

    return DictLetterFreq

### 2:提取总体的频率作为保准频率

这一步需要一个英语文字(或者其他类似语言)的范本,数据量要足够的大,下面会把这个范本里面的字符的频率提取出来作为标准频率

In [48]:
with open('english.txt', 'r') as EnglishData:
    EnglishData = EnglishData.read()
HistoramNumFreq = HistogramFreqExtract(EnglishData)

### 3:建立评价函数

In [49]:
def HistogramMatchScore(message):
    LetterFreqList = list(HistogramFreqExtract(message).values())
    HistogramFreqList = list(HistoramNumFreq.values())
    LetterFreqArray = np.asarray(LetterFreqList)
    HistogramFreqArray = np.asarray(HistogramFreqList)
    Score = sum(np.absolute(LetterFreqArray-HistogramFreqArray))
    return Score

### 4:建立分析函数

**4.1** 一些前期准备

In [50]:
# 这个函数会用于排序
def getItemAtIndexOne(x):
    return x[1]
def getItemAtIndexZero(x):
    return x[0]

之所以要排序,是因为后面建立评价函数的时候,是样本的各个符号对应的频率和总体的大数据对应符号的频率做差.所以一定是**相同符号频率的差**,所以为了统一处理需要做一个排序

In [51]:
# 建立交换key中任何两个字符位置的函数
def Swap(key,symbolA,symbolB):
    ListKey = list(key)
    ListKey[key.find(symbolA)],ListKey[key.find(symbolB)] = ListKey[key.find(symbolB)],ListKey[key.find(symbolA)]
    key = ''.join(ListKey)
    return key

下面这个函数是给密钥中的字符按照频率顺序排序,这将作为最后分析函数密钥的**初始值**

注意这个函数也是要用于bigram皮女郎分析函数当中的初始值的

In [52]:
# 这个函数是把字母按照频率由高到低进行排序的函数
# 定义这个函数有五个步骤
def FreqOrderkey(message):
    message = StandardizeText(message)
# 1.得到按照频率由高到低的排序
    letterToFreq =HistogramFreqExtract(message)
    freqPairs = list(letterToFreq.items())
    freqPairs.sort(key=getItemAtIndexOne, reverse=True)
    
    freqOrder = []
    for freqPair in freqPairs:
        freqOrder.append(freqPair[0])
        

    return ''.join(freqOrder)

In [53]:
# 建立一个密钥转换函数:把按照频率对应的密钥,即对应' etaoi...'的密钥转换成对应' abcde....'的密钥
def convertKey(freqOrder):
    keyList = freqOrder
    etaoiList = list('etaoinshrdlcumwfgypbvkjxqz')
    keyPairs = list(zip(keyList,etaoiList))
    keyPairs.sort(key = getItemAtIndexOne)
    
    key = []
    for keyPair in keyPairs:      
        key.append(keyPair[0])
    return ''.join(key)

没有比要做histogram的迭代频率分析函数,因为直接简单的分析得到的结果通过评价函数得到的分数比明文还低,所以再做循环完全是没有任何必要的

## Bigram下的频率分析

### 第一步:由大数据文件中导出bigram频率作为标准频率

**1.1**:生成所有由26个字母组成的bigram组合

In [54]:
bigramLetters = []
for letter1 in LETTERS:
    for letter2 in LETTERS:
        bigramLetters.append(letter1 + letter2) 

从text中获取其中所有bigram组合的函数

In [55]:
def GetBigram(message):
    #处理message,全部转化为小写，去掉换行，去掉非字母部分
    message = StandardizeText(message)        
    bigrams = list(ngrams(message,n=2))
    
    length = len(bigrams)
    newBigrams = []
    for i in range(length):
        bigram = bigrams[i][0] + bigrams[i][1]
        newBigrams.append(bigram)
        
    return newBigrams

### 第二步:建立能从材料得到bigram数据的函数

建立bigramCount函数,从材料中提取出bigram的频数,这一步暂且不计算频率和顺序

In [56]:
from nltk import ngrams

In [57]:
# 这个函数需要之前的那值为0的字典:bigramCountDict
# 本函数不需要排序是因为提取大数据和样本都是用的同样的bigramCountDict种的顺序
def bigramFreqExtract(message):
# 0.每次清零bigramCount字典
    bigramCount = dict(zip(bigramLetters,list(repeat(0,26 ** 2))))
# 1.得到bigrams
    bigramFromNewMessage = GetBigram(message) #这个函数中已将文字标准化
# 2.得到频数
    for bigram in bigramFromNewMessage:
        bigramCount[bigram] += 1
# 3.得到频率
# 3.1先计算总数
    AlphaNumber = 0
    for letter in message:
        if letter in LETTERS:
            AlphaNumber += 1
# 3.2计算频率
    bigramCountValues = list(bigramCount.values())
    bigramCountKeys = list(bigramCount.keys())
    bigramCountValuesArray = np.asarray(bigramCountValues)
    #和上面的bigramsNumFreq一致采用百分数
    bigramFreqArray = (bigramCountValuesArray / AlphaNumber) * 100 
    bigramFreqList = bigramFreqArray.tolist()
    #这里得到字典是为了之后按照 bigramsNumFreq对应的顺序排序
    bigramExtractDict = dict(zip(bigramCountKeys,bigramFreqList)) 
# 4.提取数据
    bigramExtractDictPairs = list(bigramExtractDict.items())
    bigramExtractNumFreq = []
    for bigramExtractDictPair in bigramExtractDictPairs:
        bigramExtractNumFreq.append(bigramExtractDictPair[1])

    return bigramExtractNumFreq


**2.1:** 用bgramFreqExtract函数导出EnglishData当中的bigram数据
EnglishData就是histogram分析里面的'MobyDick.txt','UncleTom’sCabin.txt'文字材料

In [58]:
# 先把bigram数据做好,保留下来,免得测量函数部分继续计算
bigramNumFreq = bigramFreqExtract(EnglishData)

### 第三步:建立测量函数

建立函数bigramMatchScore函数,用于对比给定材料和通常英文材料之间的差异,分数越大差异越大

In [59]:
def bigramMatchScore(message):
    bigramExtractArray = np.asarray(bigramFreqExtract(message)) # 从需要频率分析的材料中提取出来的bigram数据
    bigramDataArray = np.asarray(bigramNumFreq)                  # 从大数据文件中提取出来的bigram数据
    bigramScore = np.sum(np.absolute(bigramExtractArray-bigramDataArray))
    return bigramScore

### 第四步:建立bigramAnalysis函数

bigramAnalysis将由一个简单频率分析(Histogram)分析得到的key作为初始的key,然交换密钥中字母的顺序以得到更小的bigramScore

In [60]:
from random import randint

In [61]:
def bigramFreqAnalysis(Cipher,n):
    #这里FreqOrderkey以及MonoAlphabeticCipher均内置将文字标准化的程序，因此不需要额外处理
    FreqKey = FreqOrderkey(Cipher) #这里的key是按照频率排序来的,并不是标准格式的key,所以需要转换
    key = convertKey(FreqKey)      #转换为标准格式的key
    Translation = MonoAlphabeticCipher(key,Cipher,'decrypt')
    Score = bigramMatchScore(Translation)
# 1.按照频率对Freqkey进行交换,优化破译结果
    for b in range(1,26):
        a = 0
        while a + b <= 25:
            FreqKey0 = Swap(FreqKey,FreqKey[a],FreqKey[a + b]) #交换是按照频率顺序交换的,所以是交换Freqkey而不是标准key
            key0 = convertKey(FreqKey0)
            Translation0 = MonoAlphabeticCipher(key0,Cipher,'decrypt')
            Score0 = bigramMatchScore(Translation0)
            a += 1
            if Score0 < Score:
                FreqKey = FreqKey0
                Translation = Translation0
                Score = Score0
# 2.往往第一阶段完成以后结果还不是最好,所以接下来随机的进行替换,使得结果更好

    key = convertKey(FreqKey) #得到上一个阶段最后的密钥的标准格式
    
    for i in range(0,n):  # n是循环的次数
        RandChoiceA = randint(0,25) #得到1到26的随机整数
        RandChoiceB = randint(0,25)
        if RandChoiceA != RandChoiceB:
            key0 = Swap(key,key[RandChoiceA],key[RandChoiceB]) #随机交换密钥中两个字母的顺序
            Translation0 = MonoAlphabeticCipher(key0,Cipher,'decrypt') # 解密
            Score0 = bigramMatchScore(Translation0) # 评价
            if Score0 < Score: #如果评价更好,得到新的密钥
                key = key0
                Translation = Translation0
                Score = Score0
    #去掉所有空格
    result0 = []
    
    for letter in Translation:
        if letter.isalpha():
            result0.append(letter)
                
    return ''.join(result0)

## 测试

### 1:明文

明文不需要预处理,因为处理的函数写进了每一个Exract函数

In [62]:
PlainText = '''
In cryptography, a substitution cipher is a method of encrypting by which units of plaintext are replaced with ciphertext, according to a fixed system; the "units" may be single letters (the most common), pairs of letters, triplets of letters, mixtures of the above, and so forth. The receiver deciphers the text by performing the inverse substitution.

Substitution ciphers can be compared with transposition ciphers. In a transposition cipher, the units of the plaintext are rearranged in a different and usually quite complex order, but the units themselves are left unchanged. By contrast, in a substitution cipher, the units of the plaintext are retained in the same sequence in the ciphertext, but the units themselves are altered.

There are a number of different types of substitution cipher. If the cipher operates on single letters, it is termed a simple substitution cipher; a cipher that operates on larger groups of letters is termed polygraphic. A monoalphabetic cipher uses fixed substitution over the entire message, whereas a polyalphabetic cipher uses a number of substitutions at different positions in the message, where a unit from the plaintext is mapped to one of several possibilities in the ciphertext and vice versa. 
'''
PlainText

'\nIn cryptography, a substitution cipher is a method of encrypting by which units of plaintext are replaced with ciphertext, according to a fixed system; the "units" may be single letters (the most common), pairs of letters, triplets of letters, mixtures of the above, and so forth. The receiver deciphers the text by performing the inverse substitution.\n\nSubstitution ciphers can be compared with transposition ciphers. In a transposition cipher, the units of the plaintext are rearranged in a different and usually quite complex order, but the units themselves are left unchanged. By contrast, in a substitution cipher, the units of the plaintext are retained in the same sequence in the ciphertext, but the units themselves are altered.\n\nThere are a number of different types of substitution cipher. If the cipher operates on single letters, it is termed a simple substitution cipher; a cipher that operates on larger groups of letters is termed polygraphic. A monoalphabetic cipher uses fixe

### 2:生成随机密钥并加密

In [66]:
key = 'wykcmafinhsgbztdvjpourxleq'
Cipher = MonoAlphabeticCipher(key,PlainText,'encrypt')
print('key:%s\n' %(key))
print('Ciphertext:\n%s' %(Cipher))

key:wykcmafinhsgbztdvjpourxleq

Ciphertext:
nzkje dotfj wdiew puypo nouon tzknd imjnp wbmoi tctam zkjed onzfy exink iuzno ptadg wnzom lowjm jmdgw kmcxn oiknd imjom lowkk tjcnz fotwa nlmcp epomb oimuz nopbw eympn zfgmg moomj poimb tpokt bbtzd wnjpt agmoo mjpoj ndgmo ptagm oomjp bnlou jmpta oimwy trmwz cptat joioi mjmkm nrmjc mkndi mjpoi momlo yedmj atjbn zfoim nzrmj pmpuy ponou ontzp uypon ouont zkndi mjpkw zymkt bdwjm cxnoi ojwzp dtpno ntzkn dimjp nzwoj wzpdt pnont zkndi mjoim uznop taoim dgwnz omlow jmjmw jjwzf mcnzw cnaam jmzow zcupu wggev unomk tbdgm ltjcm jyuoo imuzn opoim bpmgr mpwjm gmaou zkiwz fmcye ktzoj wponz wpuyp onouo ntzkn dimjo imuzn optao imdgw nzoml owjmj mownz mcnzo impwb mpmvu mzkmn zoimk ndimj omloy uooim uznop oimbp mgrmp wjmwg omjmc oimjm wjmwz ubymj tacna amjmz ooedm ptapu ypono uontz kndim jnaoi mkndi mjtdm jwomp tzpnz fgmgm oomjp nonpo mjbmc wpnbd gmpuy ponou ontzk ndimj wkndi mjoiw otdmj wompt zgwjf mjfjt udpta gmoom jpnpo mjbmc dtgef jwdin kwbtz twgdi wymon kk

### 3:测试bigram频率分析

In [67]:
import time 

In [68]:
start = time.time()
bigramDecrypt = bigramFreqAnalysis(Cipher,2000)
end = time.time()

convertKey(FreqOrderkey(Cipher))
simpleDecrypt = MonoAlphabeticCipher(convertKey(FreqOrderkey(Cipher)),Cipher,'decrypt')

print('破译结果:')                   
print(bigramDecrypt)
print('破译评价:')
print('==============破译时间:%s seconds'%(end-start))
print('==============明文得分:%s ponints'%(bigramMatchScore(PlainText)))
print('==============密文得分:%s ponints'%(bigramMatchScore(Cipher)))
print('==============histogram译文得分:%s ponints'%(bigramMatchScore(simpleDecrypt)))
print('==============bigram译文得分:%s ponints'%(bigramMatchScore(bigramDecrypt)))

破译结果:
incryptographyasubstitutioncipherisamethodofencryptingbywhichunitsofplaintektarereplacedwithciphertektaccordingtoafikedsystemtheunitsmaybesinglelettersthemostcommonpairsofletterstripletsoflettersmikturesoftheaboveandsoforththereceiverdeciphersthetektbyperformingtheinversesubstitutionsubstitutioncipherscanbecomparedwithtranspositionciphersinatranspositionciphertheunitsoftheplaintektarerearrangedinadifferentandusuallyquitecomplekorderbuttheunitsthemselvesareleftunchangedbycontrastinasubstitutionciphertheunitsoftheplaintektareretainedinthesamesequenceintheciphertektbuttheunitsthemselvesarealteredthereareanumberofdifferenttypesofsubstitutioncipherifthecipheroperatesonsinglelettersitistermedasimplesubstitutioncipheracipherthatoperatesonlargergroupsoflettersistermedpolygraphicamonoalphabeticcipherusesfikedsubstitutionovertheentiremessagewhereasapolyalphabeticcipherusesanumberofsubstitutionsatdifferentpositionsinthemessagewhereaunitfromtheplaintektismappedtooneofseveralpossibilitiesinth